# Negative prompting and avoiding undesired outputs

This notebook demonstrates how to guide and constrain model output using negative prompting, a technique that involves explicitly stating what the model should avoid doing. When working with LLMs, we often think about what should be included in a response—but it's just as important to think about what shouldn't. Unwanted content can include off-topic information, overly technical language, inappropriate analogies, or even subtle biases. Our goal is to influence the tone and content of LLM responses by excluding certain behaviors or topics.

In [1]:
import os
import re
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

## Example 1: Negative prompting with instructional constraints
We begin with a simple scenario: instructing the model to explain a topic without including certain types of information. This demonstrates how we can limit output style and content through structured language in the prompt.

In [3]:
# Create a prompt template with explicit exclusions (negative instructions)
negative_example_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Provide a brief explanation of {topic}.
    Do NOT include any of the following in your explanation:
    - Technical jargon or complex terminology
    - Historical background or dates
    - Comparisons to other related topics
    Your explanation should be simple, direct, and focus only on the core concept."""
)

# Invoke the prompt with a specific topic
response = llm.invoke(negative_example_prompt.format(topic="photosynthesis")).content
print(response)

Photosynthesis is the process by which plants, algae, and some bacteria make their own food using sunlight. They take in carbon dioxide from the air and water from the soil. Using sunlight, they transform these ingredients into sugar, which serves as food, and oxygen, which they release into the air. This process is essential for life on Earth, as it provides energy for the plants and oxygen for other living beings.


We define a `PromptTemplate` that makes it clear what should be omitted from the model's output. Then we pass the formatted prompt to the model and retrieve the content. This showcases how natural language can be structured to act as a control mechanism for LLM outputs.

Here, we are prompting the model to intentionally avoid certain response patterns. The instructions help prevent over-explaining or veering off-topic.

## Example 2: Explicit dynamic exclusions
Here, we introduce variable-driven exclusions — where the content to be excluded is passed dynamically. This is useful when the same structural prompt is reused across different tasks or user preferences.

In [4]:
# Create a prompt template that accepts a custom exclusion term
exclusion_prompt = PromptTemplate(
    input_variables=["topic", "exclude"],
    template="""Write a short paragraph about {topic}.
    Important: Do not mention or reference anything related to {exclude}."""
)

# Use the prompt to request a response about exercise while excluding body image
response = llm.invoke(exclusion_prompt.format(
    topic="the benefits of exercise",
    exclude="weight loss or body image"
)).content
print(response)

Exercise offers a myriad of benefits that extend beyond physical appearance, enhancing overall well-being. Engaging in regular physical activity boosts cardiovascular health, strengthens muscles, and improves flexibility, contributing to greater functional ability in daily tasks. Additionally, exercise is known to elevate mood and reduce symptoms of anxiety and depression by releasing endorphins, the body's natural feel-good chemicals. It also promotes better sleep quality and cognitive function, fostering sharper focus and creativity. Overall, incorporating exercise into daily life cultivates a sense of vitality and resilience, enriching both mental and physical health.


We are dynamically injecting the term to exclude (`exclude`) into the prompt structure. This allows us to reuse the same prompt logic across multiple applications while adapting the filtering criteria based on context or user input. In practice, this is useful when filtering outputs based on audience sensitivity, domain-specific restrictions, or platform policy (e.g., health disclaimers).

## Example 3: Structured constraints on style and content
In this section, we apply more rigorous and structured constraints using a formatted prompt. This includes avoiding specific words, limiting word count, preventing figurative language, and enforcing factual content.

In [5]:
# Define a prompt template with multiple constraints and stylistic rules
constraint_prompt = PromptTemplate(
    input_variables=["topic", "style", "excluded_words"],
    template="""Write a {style} description of {topic}.
    Constraints:
    1. Do not use any of these words: {excluded_words}
    2. Keep the description under 100 words
    3. Do not use analogies or metaphors
    4. Focus only on factual information"""
)

# Provide topic and constraints for the LLM
response = llm.invoke(constraint_prompt.format(
    topic="artificial intelligence",
    style="technical",
    excluded_words="robot, human-like, science fiction"
)).content
print(response)

Artificial intelligence (AI) refers to the development of algorithms and computational models that enable machines to perform tasks typically requiring cognitive functions. This includes data analysis, pattern recognition, decision-making, and learning from experience through techniques such as machine learning and deep learning. AI systems utilize large datasets to improve their accuracy and efficiency. They can operate in various domains, including natural language processing, computer vision, and autonomous systems, facilitating automation and enhancing problem-solving capabilities across industries. AI technologies are continuously evolving, driven by advancements in computational power and algorithmic research.


We are layering multiple prompt-level instructions to limit creativity and enforce factual tone. This approach ensures that generated content stays within desired constraints, useful for academic, technical, or policy-driven environments.


## Evaluation: verifying adherence to constraints
Now that we have defined rules for our outputs, we need a mechanism to evaluate whether the model actually followed them. This function allows us to define constraint checks and evaluate any given output against them.

In [6]:
# Define a function that applies evaluation checks to a model response
def evaluate_output(output, constraints):
    """Evaluate if the output meets the given constraints."""
    results = {}
    for constraint, check_func in constraints.items():
        results[constraint] = check_func(output)
    return results

# Define some example constraints
constraints = {
    "word_count": lambda x: len(x.split()) <= 100,
    "no_excluded_words": lambda x: all(word not in x.lower() for word in ["robot", "human-like", "science fiction"]),
    "no_analogies": lambda x: not re.search(r"\b(as|like)\b", x, re.IGNORECASE)

}

# Apply the evaluation logic to the previous response
evaluation_results = evaluate_output(response, constraints)
print("Evaluation results:", evaluation_results)

Evaluation results: {'word_count': True, 'no_excluded_words': True, 'no_analogies': False}


Each constraint is implemented as a Python lambda function and evaluated against the output. This enables a rule-based QA pipeline where we can validate and score LLM responses — particularly useful in production or compliance-sensitive workflows.

### Refining the prompt based on evaluation results
If any constraint fails, we can make the prompt more specific and reattempt generation. This feedback loop helps iteratively improve the quality and consistency of outputs.

In [8]:
# If the original output fails validation, refine the prompt with stronger exclusions
if not all(evaluation_results.values()):
    refined_prompt = constraint_prompt.format(
        topic="artificial intelligence",
        style="technical and concise",  # Added 'concise' to address word count
        excluded_words="robot, human-like, science fiction, like, as"  # Added 'like' and 'as' to avoid analogies and that led to failures
    )
    # Get a new response with refined prompt
    refined_response = llm.invoke(refined_prompt).content
    print("\nRefined response:\n", refined_response)

    # Evaluate the refined output
    refined_evaluation = evaluate_output(refined_response, constraints)
    print("\nRefined evaluation results:", refined_evaluation)


Refined response:
 Artificial intelligence (AI) refers to the development of computer systems capable of performing tasks that typically require cognitive functions. These tasks include problem-solving, pattern recognition, natural language processing, and decision-making. AI employs algorithms, machine learning, and deep learning techniques to analyze data, adapt to new information, and improve performance over time. Applications span various domains, including healthcare, finance, transportation, and entertainment, facilitating automation and enhancing efficiency. AI systems can operate within defined parameters or learn from vast datasets to generate insights and predictions.

Refined evaluation results: {'word_count': True, 'no_excluded_words': True, 'no_analogies': True}


We close the feedback loop between model generation and evaluation. If the response violates rules (e.g., includes analogies or banned terms), we adapt the prompt accordingly—demonstrating how LLM interaction can be iterative and self-correcting.